# étude de l'article 2

In [2]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation_arcticle_1 import *
from src.modelisation_arcticle_2 import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


In [3]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

In [4]:
df_modelisation = read_and_prepare_df_of_the_model('01. output/df_freqs_speaker_word.csv')

In [5]:
df_modelisation['m_it'] = df_modelisation.groupby(by=['Speaker', 'party'])['c_ijt'].transform('sum')

In [6]:
# On Calcule les termes ds le sigma 
df_modelisation['q_hat_it'] = df_modelisation['c_ijt']/df_modelisation['m_it']

In [7]:
df_modelisation_Lab = df_modelisation.loc[df_modelisation['party'] == 'Lab']
df_modelisation_Con = df_modelisation.loc[df_modelisation['party'] == 'Con']

## plug-in estimator (biased)

In [8]:
df_modelisation_Lab_plugin = plugin_estimator_by_party(df_modelisation_Lab, 'Lab')
df_modelisation_Con_plugin = plugin_estimator_by_party(df_modelisation_Con, 'Con')

In [9]:
df_modelisation_plugin = plugin_estimator(
    df_modelisation_Lab_plugin,
    df_modelisation_Con_plugin
)

In [10]:
# we compute pi
pi = compute_pi_plugin(df_modelisation_plugin)
pi

0.5055121348240272

## leave-out estimator (not biased)

In [11]:
# We compute the q_ijt_column
df_modelisation_Con_leave_out = compute_q_ijt_column(df_modelisation_Con, 'Con')
df_modelisation_Lab_leave_out  = compute_q_ijt_column(df_modelisation_Con, 'Lab')

In [12]:
# La je fais mon merge 
df_modelisation_Con_leave_out = pd.merge(
    df_modelisation_Con_leave_out,
    df_modelisation_Lab_plugin,
    how='left',
    on=['variable']
)
df_modelisation_Con_leave_out['rho_hat_-ijt'] = (df_modelisation_Con_leave_out['q_hat_Con_-ijt']) / (df_modelisation_Con_leave_out['q_hat_Con_-ijt'] + df_modelisation_Con_leave_out['q_hat_Lab_jt'])
df_modelisation_Con_leave_out['produit'] = df_modelisation_Con_leave_out['q_hat_it'] * df_modelisation_Con_leave_out['rho_hat_-ijt']
nb_Con = len(df_modelisation_Con_leave_out['Speaker'].unique())
res_Con = df_modelisation_Con_leave_out[['variable', 'produit']].groupby(by=['variable']).sum().reset_index()
res_Con['moitie_Con_pi'] = res_Con['produit']*(1/2)*(1/nb_Con)
res_Con.drop(columns=['produit'], inplace=True)

In [80]:
#  La je fais mon merge 
df_modelisation_Lab_leave_out = pd.merge(
    df_modelisation_Lab_leave_out,
    df_modelisation_Con_plugin,
    how='left',
    on=['variable']
)
df_modelisation_Lab_leave_out['rho_hat_-ijt'] = (df_modelisation_Lab_leave_out['q_hat_Lab_-ijt']) / (df_modelisation_Lab_leave_out['q_hat_Lab_-ijt'] + df_modelisation_Lab_leave_out['q_hat_Con_jt'])
df_modelisation_Lab_leave_out['produit'] = df_modelisation_Lab_leave_out['q_hat_it'] * df_modelisation_Lab_leave_out['rho_hat_-ijt']
nb_Lab = len(df_modelisation_Lab_leave_out['Speaker'].unique())
res_Lab = df_modelisation_Lab_leave_out[['variable', 'produit']].groupby(by=['variable']).sum().reset_index()
res_Lab['moitie_Lab_pi'] = res_Lab['produit']*(1/2)*(1/nb_Lab)
res_Lab.drop(columns=['produit'], inplace=True)

In [81]:
res = pd.merge(
    res_Con,
    res_Lab,
    how='left',
    on=['variable'],
)

In [82]:
res['pi'] = res['moitie_Con_pi'] + res['moitie_Lab_pi']

In [84]:
res['pi'].sum()

0.5008476141903633

## Test sur un speaker

In [85]:
pi_speaker_Con = df_modelisation_Con[['Speaker','produit']].groupby(by=['Speaker']).sum().reset_index()


KeyError: "['produit'] not in index"

In [ ]:
# Pour les Con
pi_speaker_Con = df_modelisation_Con[['Speaker','produit']].groupby(by=['Speaker']).sum().reset_index()
pi_speaker_Con['party'] = 'Con'

# Pour les Lab
pi_speaker_Lab = df_modelisation_Lab[['Speaker','produit']].groupby(by=['Speaker']).sum().reset_index()
pi_speaker_Lab['party'] = 'Lab'


In [ ]:
pi_speaker = pd.concat([pi_speaker_Con, pi_speaker_Lab])
pi_speaker.rename(columns={'produit':'speaker_pi'},inplace=True)
pi_speaker = pi_speaker.loc[pi_speaker['speaker_pi'] > 0]

In [ ]:
import plotly.express as px
fig = px.scatter(pi_speaker, y="party", x="speaker_pi", color="party", symbol="party")
fig.update_traces(marker_size=10)
fig.show()

## avec les articles 

In [ ]:
df_Guardian = pd.read_csv('00. input/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('00. input/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

Il faut d'abord effectuer un traitement similaire df_modelisation articles des journdf_modelisation

In [ ]:
def treat_1(df, party):
  df = df[['author', 'fulltext']]
  if party == 'Guardian' :
    df['party'] = 'Lab'
  if party == 'DE' :
    df['party'] = 'Con'
  df.rename(columns=
          {'author' : 'Speaker', 'fulltext':'text'}, inplace=True)
  return df

In [ ]:
def treat_2(df_1,df_2):
  df = pd.concat([df_1, df_2])
  df.dropna().reset_index(drop=True)
  return df

In [ ]:
def treat_3(df):
  df['text'] = df['text'].apply(clean, args=(stop_words,))
  return df 

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')

In [ ]:
df_DE = treat_1(df_DE, 'DE')

In [ ]:
df_newspaper = treat_2(df_Guardian, df_DE)

In [ ]:
df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
df_newspaper = df_newspaper.head(1000)

In [ ]:
df_newspaper = treat_3(df_newspaper)

NameError: name 'stop_words' is not defined

In [ ]:
df_newspaper_treated = df_newspaper

In [ ]:
df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()

In [ ]:
df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')

In [ ]:
df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)

In [ ]:
list_of_words = selected_words(df_freqs)

In [ ]:
df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)

In [ ]:
df_newspaper_treated['Unnamed: 0'] = 0

In [ ]:
LO_treatment(df_newspaper_treated)

In [ ]:
def treat_fonction(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  return LO_treatment(df_newspaper_treated)

In [ ]:
treat_fonction(df_Guardian, df_DE)

# **Évolution de la polarisation dans le temps**

In [ ]:
df_Guardian = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/TheGuardian/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/DailyExpress/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

In [ ]:
def treat_1_date(df, party, year):
  df = df.loc[df['year'] == year]
  df = df[['author', 'fulltext']]
  if party == 'Guardian' :
    df['party'] = 'Lab'
  if party == 'DE' :
    df['party'] = 'Con'
  df.rename(columns=
          {'author' : 'Speaker', 'fulltext':'text'}, inplace=True)
  return df

In [ ]:
def treat_fonction(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  print('SORTIE')
  return LO_treatment(df_newspaper_treated)

In [ ]:
df_1 = treat_1_date(df_Guardian, 'Guardian', 2020)
df_2 = treat_1_date(df_DE, 'DE', 2020)

In [ ]:
treat_fonction(df_1, df_2)

In [ ]:
evolution_of_pi = []

In [ ]:
for year in range(2010, 2023 ):
  df_a = treat_1_date(df_Guardian, 'Guardian', year)
  df_b = treat_1_date(df_DE, 'DE', year)
  pi = treat_fonction(df_a, df_b)
  evolution_of_pi.append(pi)
  print(evolution_of_pi, 'BOUCLE')

In [ ]:
evolution_of_pi

In [ ]:
plt.plot([i for i in range(2010, 2023)], evolution_of_pi)
plt.show()

# **Partisanship des mots**

In [ ]:
df_Guardian = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/TheGuardian/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/DailyExpress/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')
df_DE = treat_1(df_DE, 'DE')

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')
df_DE = treat_1(df_DE, 'DE')

In [ ]:
def treat_table(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  return LO_treatment_table(df_newspaper_treated)

In [ ]:
table_Con, table_Lab = treat_table(df_Guardian, df_DE)

In [ ]:
table_Con.dropna().reset_index(drop=True)
table_Lab.dropna().reset_index(drop=True)